## MOONRAKER HACK 6124
## Speach Fact Checker
### Using LangGraph - Agent Chain with multiple tools

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

### Dependencies
. . .

In [1]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search tavily-python

### Environment Variables
OpenAI API, Tavily and LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE2 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

### Agent Tool Belt

Adding tools that equip our agent with a toolbelt to help answer questions and add external knowledge.

In [5]:
#from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults

tool_belt = [
#    DuckDuckGoSearchRun(),
    TavilySearchResults(), # TavilyRun() replaced duckduckgo-search because of rate limiting
    ArxivQueryRun() # Maybe use a different tool here for sentiment and all
]

### Actioning with Tools
The Agent will use a ToolExecutor 

In [6]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Using OpenAI LLM, leveraging the OpenAI function calling API.

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

OK, let's "put on the tool belt"

In [8]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

### The Agent State Machine
LangGraph StatefulGraph - AgentState object.

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [9]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

### Build the Graph

In [10]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

In [13]:
# AI MAKERSPACE PREPR 
# Date: 2024-5-16

# Basic Imports & Setup
# import os
# from openai import AsyncOpenAI

from langchain.agents import Tool

# Using Chainlit for our UI
# import chainlit as cl
# from chainlit.prompt import Prompt, PromptMessage
# from chainlit.playground.providers import ChatOpenAI

# Getting the API key from the .env file
from dotenv import load_dotenv
load_dotenv()

# RAG pipeline imports and setup code
# Get the DeveloperWeek PDF file (future implementation: direct download from URL)
from langchain.document_loaders import PyMuPDFLoader

# Adjust the URL to the direct download format
file_id = "1JeA-w4kvbI3GHk9Dh_j19_Q0JUDE7hse"
direct_url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Now load the document using the direct URL
docs = PyMuPDFLoader(direct_url).load()

import tiktoken
def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

# Split the document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,           # 500 tokens per chunk, experiment with this value
    chunk_overlap = 50,        # 50 tokens overlap between chunks, experiment with this value
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

# Load the embeddings model
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Load the vector store and retriever from Qdrant
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="Prepr",
)

qdrant_retriever = qdrant_vectorstore.as_retriever()

from langchain_openai import ChatOpenAI
openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """
SYSTEM:
You are a professional personal assistant.
You are a helpful personal assistant who provides information about conferences.
You like to provide helpful responses to busy professionals who ask questions about conferences.

You can have a long conversation with the user about conferences.
When to talk with the user about conferences, it can be a "transactional conversation" with a prompt-response format with one prompt from the user followed by a response by you.

Here is an example of a transactional conversation:
User: When is the conference?
You: The conference is on June 1st, 2024. What else would you like to know?

It can also be a chain of questions and answers where you and the user continues the chain until they say "Got it".
Here is an example of a transactional conversation:
User: What sessions should I attend?
You: You should attend the keynote session by Bono. Would you like to know more?
User: Yes
You: The keynote session by Bono is on June 1st, 2024. What else would you like?

If asked a question about a sessions, you can provide detailed information about the session.
If there are multiple sessions, you can provide information about each session.

The format of session related replies is:
Title:
Description:
Speaker:
Background:
Date:
Topics to Be Covered:
Questions to Ask:

CONTEXT:
{context}

QUERY:
{question}
Most questions are about the date, location, and purpose of the conference.
You may be asked for fine details about the conference regarding the speakers, sponsors, and attendees.
You are capable of looking up information and providing detailed responses.
When asked a question about a conference, you should provide a detailed response.
After completing your response, you should ask the user if they would like to know more about the conference by asking "Hope that helps".
If the user says "yes", you should provide more information about the conference. If the user says "no", you should say "Goodbye! or ask if they would like to provide feedback.
If you are asked a question about Cher, you should respond with "Rock on With Your Bad Self!".
If you can not answer the question, you should say "I am sorry, I do not have that information, but I am always here to help you with any other questions you may have.".
"""
rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

retrieval_augmented_qa_chain.invoke({"question": 'whens the event coming?'})

rag_tool = Tool(
    name="RAGTool",
    func=lambda inputs: retrieval_augmented_qa_chain.invoke(inputs),
    description="Use this tool to answer questions using the RAG approach."
)

from langgraph.graph import StateGraph, END

# Define a state schema
state_schema = {
    "start": "agent",
    "states": {
        "agent": {
            "next": "action",
            "type": "function",
            "function": lambda context: {"input": context["input"]}
        },
        "action": {
            "next": END,
            "type": "tool",
            "tool": rag_tool
        }
    }
}

Now we have two total nodes. We have:
- `call_model` is a node that will...well...call the model
- `call_tool` is a node which will call a tool

In [14]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(state_schema=state_schema)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

/Users/mcshrub/miniconda3/envs/mikec-aim/lib/python3.11/site-packages/langgraph/graph/state.py:45: UserWarning: Invalid state_schema: {'start': 'agent', 'states': {'agent': {'next': 'action', 'type': 'function', 'function': <function <lambda> at 0x12aa90900>}, 'action': {'next': '__end__', 'type': 'tool', 'tool': Tool(name='RAGTool', description='Use this tool to answer questions using the RAG approach.', func=<function <lambda> at 0x12aa90540>)}}}. Expected a type or Annotated[type, reducer]. Please provide a valid schema to ensure correct updates.
 See: https://langchain-ai.github.io/langgraph/reference/graphs/#stategraph
  warnings.warn(


Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [15]:
workflow.set_entry_point("agent")

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow. We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

In [16]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [17]:
workflow.add_edge("action", "agent")

All that's left to do now is to compile our workflow - and we're off!

In [18]:
app = workflow.compile()

#### Helper Function to print messages

In [19]:
def print_messages(messages):
  next_is_tool = False
  initial_query = True
  for message in messages["messages"]:
    if "function_call" in message.additional_kwargs:
      print()
      print(f'Tool Call - Name: {message.additional_kwargs["function_call"]["name"]} + Query: {message.additional_kwargs["function_call"]["arguments"]}')
      next_is_tool = True
      continue
    if next_is_tool:
      print(f"Tool Response: {message.content}")
      next_is_tool = False
      continue
    if initial_query:
      print(f"Initial Query: {message.content}")
      print()
      initial_query = False
      continue
    print()
    print(f"Agent Response: {message.content}")


## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!


In [22]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="When is DeveloperWeek conference scheduled?")]}
#inputs = {"messages" : [HumanMessage(content="What is RAG?")]}
messages = app.invoke(inputs)

print_messages(messages)

Initial Query: I found some information about the DeveloperWeek 2024 conference. Here are some details:

1. **Conference Schedule**: DeveloperWeek 2024 will take place from February 21-29, 2024, in the San Francisco Bay Area and virtually. It is described as the world's largest developer and engineering technology conference and expo. You can register for the event on their [official website](https://www.developerweek.com/conference/).

2. **Event Details**: The conference will include various activities such as workshops, hackathons, expo, conference sessions, awards ceremony, and networking opportunities. The schedule is packed with different events each day from February 21-29, 2024. You can find the full schedule on their [events page](https://www.developerweek.com/events/).

3. **Virtual Participation**: For those attending virtually, the conference will have online workshops, virtual expo, and conference sessions from February 27-29, 2024. You can find more details on the [Develo

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [24]:
inputs = {"messages" : [HumanMessage(content="When did Al Gore created the Internet.  What year did Trump go to jail?")]}

messages = app.invoke(inputs)

print_messages(messages)

Initial Query: Here are some search results related to Al Gore and the invention of the Internet:

1. [The Fact Checker - A cautionary tale for politicians: Al Gore and the invention of the Internet](https://www.washingtonpost.com/news/fact-checker/wp/2013/11/04/a-cautionary-tale-for-politicians-al-gore-and-the-invention-of-the-internet/): This article discusses the claim made by Al Gore about inventing the Internet.

2. [Mental Floss - History of the U.S.: Al Gore Really Did "Take the Initiative in Creating the Internet"](https://www.mentalfloss.com/article/25986/history-us-al-gore-really-did-take-initiative-creating-internet): An article exploring Al Gore's role in the creation of the Internet.

3. [Origins - Gore Did Help Invent Internet](https://origins.osu.edu/history-news/gore-did-help-invent-internet?language_content_entity=en): This article provides a historical perspective on Al Gore's involvement in the development of the Internet.

4. [Wikipedia - Al Gore and Information Tec

### EXTRA 
### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [20]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | app | parse_output

In [21]:
agent_chain.invoke({"question" : "What is RAG for LLM Applications"})

'RAG (Retrieval Augmented Generation) for LLM (Large Language Models) applications allows us to give foundational models local context without expensive fine-tuning. It can be done on normal everyday machines like laptops. If you are interested in getting started with RAG for LLM-powered applications, you can refer to tutorials and guides available online:\n\n1. [LLM RAG Tutorial](https://colab.research.google.com/github/SamHollings/llm_tutorial/blob/main/llm_tutorial_rag.ipynb): This tutorial provides a simple introduction to getting started with an LLM to create a RAG app.\n\n2. [Your Guide to Starting With RAG for LLM-Powered Applications](https://medium.com/@caldhubaib/your-guide-to-starting-with-rag-for-llm-powered-applications-ee5ce31cab71): This guide offers insights and tips on building enterprise-grade LLMs and getting started with RAG for LLMs.\n\n3. [Basic Conversational AI with RAG Solutions](https://github.com/zahaby/intro-llm-rag): This guide is for technical teams intere